# Send Scoring Requests to Deployment (using Virtualized Asset as Source)
This notebook sends scoring requests to a deployed model, using virtualized data as the source.  You will fill in the parameters into the first cell and then run the remainder of the notebook line by line.

### Assumption:
1. Model has been deployed in Watson Machine Learning
2. The SCORING_DATA virtualized data asset is in your project, along with the connection called 'DV Connection'.  Both of these must be in the project.

### 1. Fill in the REST endpoint for your WML deployment:

In [13]:
import os,requests,json
# Endpoint for your deployed model
os.environ['ML_ENDPOINT']=" "



### 2. Run the cell below

In [14]:
os.environ['DV_SCORING_ASSET_NAME'] = "SCORING_DATA"
os.environ['DV_CONNECTION_NAME']= "DV Connection"
ml_endpoint = os.environ.get('ML_ENDPOINT').replace("\"","")
dv_connection_name = os.environ.get('DV_CONNECTION_NAME').replace("\"","")
scoring_asset_name = os.environ.get('DV_SCORING_ASSET_NAME').replace("\"","")

### 3. Run the following cell.  This will prepare the query to read the scoring data from DV.

In [15]:
# @hidden_cell

from project_lib import Project
project = Project.access()

DV_Connection__Service_Level__metadata = project.get_connection(name=dv_connection_name)

import os, ibm_db, ibm_db_dbi as dbi, pandas as pd

DV_Connection__Service_Level__dsn = 'DATABASE={};HOSTNAME={};PORT={};PROTOCOL=TCPIP;UID={uid};PWD={pwd}'.format(
    DV_Connection__Service_Level__metadata['database'],
    DV_Connection__Service_Level__metadata['host'],
    DV_Connection__Service_Level__metadata.get('port', 32051),
    uid=DV_Connection__Service_Level__metadata['username'],
    pwd=DV_Connection__Service_Level__metadata['password']
)

DV_Connection__Service_Level__connection = dbi.connect(DV_Connection__Service_Level__dsn)
   

query = 'SELECT * FROM "ADMIN"."SCORING_DATA"'

# After use, close the database connection with the following code:
# DV_Connection__Service_Level__connection.close()


### 4. Run the following cell. This will query the DV scoring data in batches of size n=100 and send scoring requests to your deployment.

In [10]:
# create chunks of 100 input requests at a time
batch_size = 100
final_pred_df = pd.DataFrame()
# NOTE: if not running as a job but running inline, best to print the iteration # in the loop to show that the loop is doing something
# pd.read_csv(data_path, chunksize=batch_size)
for data_subset in pd.read_sql_query(query, con=DV_Connection__Service_Level__connection,chunksize=batch_size) :

    # We check if there is a column with the score and remove it if so
    has_score = 'DELAYED' in (data_subset.columns.tolist())

    fields = data_subset.columns.tolist()
    
    values = data_subset.values.tolist()
    
    # Sending requests using HTTP REST API
    
    mltoken = os.environ['USER_ACCESS_TOKEN']     # user access token
    ml_endpoint = os.environ['ML_ENDPOINT']       # deployed model endpoint
    
    # http header
    header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

    # Build the input data
    payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

    # call the REST API
    response_scoring = requests.post(ml_endpoint, \
                                         json=payload_scoring, headers=header, verify=False)

    prediction_data = json.loads(response_scoring.text)
    
    response_data = prediction_data["predictions"][0]
    
    cols = response_data["fields"]
    vals = response_data["values"]
    
    # If the original data had a score column, we also append this
    pred_df = pd.DataFrame(vals, columns=cols)
    

    # append the results to the final prediction dataframe
    final_pred_df = final_pred_df.append(pred_df, ignore_index=True)
    
    # sleep for 10 secs before starting the next chunk
    # time.sleep(10)

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.ibmcloud-roks-sh4fdyvr-6ccd7f378ae819553d37d5f2ee142bd6-0000.eu-gb.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.ibmcloud-roks-sh4fdyvr-6ccd7f378ae819553d37d5f2ee142bd6-0000.eu-gb.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS requ

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.ibmcloud-roks-sh4fdyvr-6ccd7f378ae819553d37d5f2ee142bd6-0000.eu-gb.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zen-cpd-zen.ibmcloud-roks-sh4fdyvr-6ccd7f378ae819553d37d5f2ee142bd6-0000.eu-gb.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS requ

## Optionally, print the predictions:

In [11]:
final_pred_df.head()

,prediction,probability
0,0,"[0.9709695564930344, 0.029030443506965563]"
1,1,"[0.00928879062951704, 0.990711209370483]"
2,0,"[0.5396584632097075, 0.4603415367902925]"
3,0,"[0.9821631107323442, 0.017836889267655857]"
4,0,"[0.9892082796132827, 0.010791720386717262]"


In [16]:
DV_Connection__Service_Level__connection.close()

True